In [26]:
import pandas as pd
import os 
print(os.getcwd())
df_source = pd.read_csv("data/transcripts/whole_data_2021_2016.csv")

/Users/ewenmichel/Desktop/Centrale/3A/SDI/Project/Multimodal-CSC


In [27]:
for label in ['SV_Tutor', 'SV_Tutee', 'PR_Tutor', 'PR_Tutee', 'SD_Tutor', 'SD_Tutee']:
    print(df_source[label].unique())

[nan 'SV']
[nan 'SV']
[nan '0' 'UL' 'LPA' 'LP' 'LPP' 'not sure if this is praise']
[nan 'LPA' 'UL' 'LPP' 'LPB' '0' 'LP']
[nan 'SD' 'QE']
[nan 'SD' 'QE']


In [28]:
# df_source["Time_begin_test"] = df_source["Time_begin"].apply(lambda x : pd.to_datetime(x, format = "%M:%S.%f")).apply(lambda x : str(x)[11:])
# ts.strftime('%Y-%m-%d %X')

In [29]:
df_source = pd.read_csv("data/transcripts/whole_data_2021_2016.csv")

sd_df = df_source[(df_source["SD_Tutor"] == "SD") | (df_source["SD_Tutee"] == "SD")].replace("SD", "x").fillna("")
qe_df = df_source[(df_source["SD_Tutor"] == "QE") | (df_source["SD_Tutee"] == "QE")].replace("QE", "x").fillna("")
sv_df = df_source[(df_source["SV_Tutor"] == "SV") | (df_source["SV_Tutee"] == "SV")].replace("SV", "x").fillna("")
pr_df = df_source[(df_source["PR_Tutor"].isin(['LPA', 'UL', 'LPP', 'LPB', '0', 'LP'])) | (df_source["PR_Tutee"].isin(['LPA', 'UL', 'LPP', 'LPB', '0', 'LP']))].replace(['LPA', 'UL', 'LPP', 'LPB', '0', 'LP'], "x").fillna("")

sd_df["SD"] = sd_df.SD_Tutor + sd_df.SD_Tutee
qe_df["QE"] = qe_df.SD_Tutor + qe_df.SD_Tutee
sv_df["SV"] = sv_df.SV_Tutor + sv_df.SV_Tutee
pr_df["PR"] = pr_df.PR_Tutor + pr_df.PR_Tutee

sd_map_dict = dict(zip(sd_df.index, sd_df["SD"]))
qe_map_dict = dict(zip(qe_df.index, qe_df["QE"]))
sv_map_dict = dict(zip(sv_df.index, sv_df["SV"]))
pr_map_dict = dict(zip(pr_df.index, pr_df["PR"]))

df_source["SD"] = df_source.index.to_series().map(sd_map_dict)
df_source["QE"] = df_source.index.to_series().map(qe_map_dict)
df_source["SV"] = df_source.index.to_series().map(sv_map_dict)
df_source["PR"] = df_source.index.to_series().map(pr_map_dict)

df_source = df_source.drop(df_source[df_source["SV"] == "xx"].index)
df_source["SD"].unique(), df_source["QE"].unique(), df_source["SV"].unique(), df_source["PR"].unique()

(array([nan, 'x'], dtype=object),
 array([nan, 'x'], dtype=object),
 array([nan, 'x'], dtype=object),
 array([nan, 'x'], dtype=object))

In [ ]:
pd.merge(df_source,     , on = ['Dyad', 'Session', 'Period', 'P1', 'P2'])

In [60]:
from fuzzywuzzy import fuzz
from fuzzywuzzy import process

compare_P1 = pd.MultiIndex.from_product([df_source['P1'], df_target['P1']]).to_series()
compare_P2 = pd.MultiIndex.from_product([df_source['P2'], df_target['P2']]).to_series()
compare_D = pd.MultiIndex.from_product([df_source['D'], df_target['D']]).to_series()
compare_S = pd.MultiIndex.from_product([df_source['S'], df_target['S']]).to_series()
compare_S = pd.MultiIndex.from_product([df_source['P1'], df_target['P1']]).to_series()
compare_S = pd.MultiIndex.from_product([df_source['P1'], df_target['P1']]).to_series()

comparisons = [compare_P1, compare_P2, compare_D, compare_S]

def metrics(tup):
    return pd.Series([fuzz.ratio(*tup),
                      fuzz.token_sort_ratio(*tup)],
                     ['ratio', 'token'])``

for comp_ in comparisons:
    comp_.apply(metrics)
    comp_.apply(metrics).unstack().idxmax().unstack(0)
    comp_.apply(metrics).unstack(0).idxmax().unstack(0)

ModuleNotFoundError: No module named 'fuzzywuzzy'